## [Probabilistic Topic Modeling](http://pyro.ai/examples/prodlda.html#Probabilistic-Topic-Modeling)

#### Topic models are a suite of unsupervised learning algorithms that aim to discover and annotate large archives of documents with thematic information. Probabilistic topic models use statistical methods to analyze the words in each text to discover common themes, how those themes are connected to each other, and how they change over time. They enable us to organize and summarize electronic archives at a scale that would be impossible with human annotation alone. The most popular topic model is called latent Dirichlet allocation, or LDA.

![](https://i.ibb.co/zV5rjX6/Screen-Shot-2020-09-24-at-11-21-38.png)